# Ejercicios UD03_02

## Clasificar preguntas

En la práctica [Clasificación de texto con PyTorch](https://colab.research.google.com/github/martinezpenya/MIA-IABD-2425/blob/main/UD03/notebooks/2.-classificacio_text_torch_ES.ipynb) hemos visto el proceso para convertir un texto en una representación numérica que pueda ser utilizada por un algoritmo de aprendizaje automático. Hemos visto diferentes representaciones como *Bolsa de palabras* (BoW) y *incrustaciones de palabras* (word embeddings) y cómo entrenar una red neuronal para clasificar texto.

En esta práctica, deberá repetir el proceso para clasificar las preguntas en temas. Usaremos el conjunto de datos `Trec` que contiene preguntas en inglés y su tema. El conjunto de datos está disponible en [trec](https://huggingface.co/datasets/CogComp/trec).

### Objetivos de la práctica
* Reproducir el proceso visto en la práctica [Clasificación de texto con PyTorch](https://colab.research.google.com/github/martinezpenya/MIA-IABD-2425/blob/main/UD03/notebooks/2.-classificacio_text_torch_ES.ipynb) para clasificar preguntas en temáticas.
* Deberá preparar una red neuronal con PyTorch para clasificar las preguntas.
* Pruebe las diferentes representaciones vistas para convertir el texto en una representación numérica.
* Tendrá que comparar los resultados obtenidos con las diferentes representaciones.

In [198]:
# Instalamos las librerías necesarias en las versiones correctas

# %pip install --upgrade torch datasets scikit-learn transformers

In [199]:
from datasets import load_dataset

# Cargamos el conjunto de datos. Se descargará y almacenará automáticamente en local.
# Este conjunto de datos contiene noticias de diferentes categorías. En este caso
# usaremos las categorías de mundo, deportes, negocios y ciencia ficción/tecnología.

dataset = load_dataset('trec')

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'coarse_label', 'fine_label'],
        num_rows: 5452
    })
    test: Dataset({
        features: ['text', 'coarse_label', 'fine_label'],
        num_rows: 500
    })
})

In [200]:
print(dataset['train'][0])

print(dataset['train'].features)

classes = dataset['train'].features["coarse_label"].names
classes

{'text': 'How did serfdom develop in and then leave Russia ?', 'coarse_label': 2, 'fine_label': 26}
{'text': Value(dtype='string', id=None), 'coarse_label': ClassLabel(names=['ABBR', 'ENTY', 'DESC', 'HUM', 'LOC', 'NUM'], id=None), 'fine_label': ClassLabel(names=['ABBR:abb', 'ABBR:exp', 'ENTY:animal', 'ENTY:body', 'ENTY:color', 'ENTY:cremat', 'ENTY:currency', 'ENTY:dismed', 'ENTY:event', 'ENTY:food', 'ENTY:instru', 'ENTY:lang', 'ENTY:letter', 'ENTY:other', 'ENTY:plant', 'ENTY:product', 'ENTY:religion', 'ENTY:sport', 'ENTY:substance', 'ENTY:symbol', 'ENTY:techmeth', 'ENTY:termeq', 'ENTY:veh', 'ENTY:word', 'DESC:def', 'DESC:desc', 'DESC:manner', 'DESC:reason', 'HUM:gr', 'HUM:ind', 'HUM:title', 'HUM:desc', 'LOC:city', 'LOC:country', 'LOC:mount', 'LOC:other', 'LOC:state', 'NUM:code', 'NUM:count', 'NUM:date', 'NUM:dist', 'NUM:money', 'NUM:ord', 'NUM:other', 'NUM:period', 'NUM:perc', 'NUM:speed', 'NUM:temp', 'NUM:volsize', 'NUM:weight'], id=None)}


['ABBR', 'ENTY', 'DESC', 'HUM', 'LOC', 'NUM']

In [201]:
# Separar el conjunto de datos en entrenamiento y test
ds_train = dataset['train']
ds_test = dataset['test']
# Veamos cuántos ejemplos hay en cada set
print('Número de ejemplos de train:', len(ds_train))
print('Número de ejemplos de test:', len(ds_test))

Número de ejemplos de train: 5452
Número de ejemplos de test: 500


In [202]:
# Imprimimos los primeros 5 ejemplos del conjunto de entrenamiento
for w in ds_train.take(5):
    print(f"{w['coarse_label']} ({classes[w['coarse_label']]}) -> {w['text']}")

2 (DESC) -> How did serfdom develop in and then leave Russia ?
1 (ENTY) -> What films featured the character Popeye Doyle ?
2 (DESC) -> How can I find a list of celebrities ' real names ?
1 (ENTY) -> What fowl grabs the spotlight after the Chinese Year of the Monkey ?
0 (ABBR) -> What is the full form of .com ?


In [203]:
# Utilizamos el tokenizador de Bert (uno de los primeros modelos de lenguaje basados ​​en transformación) para tokenizar las oraciones
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")

# Podiamos ver el vocabulario de tokenización
vocab = tokenizer.get_vocab()
print(len(vocab))

30522


Funcion para convertir nuestra cadena tokenizada a números

In [204]:
def encode(text):
    tk = tokenizer.tokenize(text)
    return tokenizer.convert_tokens_to_ids(tk)

## BoW
Funcion para calcular el vector BoW de un comentario del dataset

In [205]:
import torch

len_vocab = len(vocab)

def to_bow(text, tamany_vocabulari=len_vocab):
    res = torch.zeros(tamany_vocabulari, dtype=torch.float32)

    for i in encode(text):
        if i<tamany_vocabulari:
            res[i] += 1
    return res

print(ds_train[0])
print(to_bow(ds_train[0]["text"]))

{'text': 'How did serfdom develop in and then leave Russia ?', 'coarse_label': 2, 'fine_label': 26}
tensor([0., 0., 0.,  ..., 0., 0., 0.])


Función que convierte las palabras textuales en tensores BoW

In [206]:
from torch.utils.data import DataLoader

def bowify(batch):
    etiquetas = torch.LongTensor([comentario["coarse_label"] for comentario in batch])
    comentarios = torch.stack([to_bow(comentario["text"]) for comentario in batch])

    return (
            etiquetas,
            comentarios
    )

train_loader = DataLoader(ds_train, batch_size=16, collate_fn=bowify)
test_loader = DataLoader(ds_test, batch_size=16, collate_fn=bowify)

## Red neuronal
El tamaño del vector de entrada es el tamaño del vocabulario, el tamaño de salida corresponde con el número de clases, en este caso 6

In [207]:
net = torch.nn.Sequential(
    torch.nn.Linear(len(vocab), 6),
    torch.nn.LogSoftmax(dim=1)
)

## Entrenamiento del modelo

In [208]:
def train_epoch(
    net,
    dataloader,
    lr=0.01,
    optimizer=None,
    loss_fn=torch.nn.NLLLoss(),
    epoch_size=None,
    report_freq=50
):

    # Si no se especifica un optimizador, usamos Adam
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr=lr)

    # Ponemos la red en modo de entrenamiento.Esto activa el comportamiento de las capas de DropOut, por ejemplo.
    net.train()

    # Inicializar las variables que nos servirán para calcular la precisión
    total_loss, acc, count, i = 0, 0, 0, 0

    # Iteremamos sobre el dataloader
    for labels, features in dataloader:

        # Ponemos los gradientes a cero
        optimizer.zero_grad()

        # calculamos la salida de la red
        out = net(features)

        # Calculamos la pérdida. Esta función ya se aplica a Softmax a la salida.
        loss = loss_fn(out, labels)  # cross_entropy(out,labels)

        # Propagamos la pérdida de regreso. Esto hará que se calculen los gradientes .
        loss.backward()

        # Actualizamos los pesos de la red. Esto toma un paso de optimización.
        optimizer.step()

        # Actualizamos variables para calcular la precisión.
        total_loss += loss

        # Calculamos la precisión. Para hacer esto, debemos convertir la salida de red en etiquetas.
        # La clase con la mayor probabilidad es la que predecimos como etiqueta.
        _, predicted = torch.max(out, 1)
        acc += (predicted == labels).sum()

        # Actualizamos el contador de muestras
        count += len(labels)

        # Mostramos la precisión cada report_freq muestras
        i += 1
        if i % report_freq == 0:
            print(f"{count}: acc={acc.item()/count}")

        # Si se especifica epoch_size y ya hemos procesado este número de muestras, dejamos el bucle.
        if epoch_size and count > epoch_size:
            break
    return total_loss.item() / count, acc.item() / count


train_epoch(net, train_loader, epoch_size=5452)


800: acc=0.4725
1600: acc=0.5575
2400: acc=0.6033333333333334
3200: acc=0.6375
4000: acc=0.665
4800: acc=0.68375


(0.059885379066383325, 0.6975421863536317)

## Words2Vec

In [193]:
import gensim.downloader as api

w2v = api.load('word2vec-google-news-300')

## Clasificación Word2Vec
Función que recibe un texto y devuelve un vector con la representación w2v del texto

In [194]:
def to_w2v(text):
    res = torch.zeros(300, dtype=torch.float32)
    for word in text:
        if word in w2v:
            res += torch.tensor(w2v.get_vector(word))
    return res

print(to_w2v(ds_train[0]["text"]))

tensor([-4.8299e+00,  4.6221e+00,  6.3916e-01,  4.6835e+00, -1.8214e+00,
         1.2753e+00, -3.7478e+00, -1.9570e+00, -2.1226e+00,  1.1957e+00,
        -1.4957e+00, -3.6926e+00, -6.9828e+00,  1.2891e+00, -5.4214e+00,
         2.3289e+00,  6.3632e+00,  6.8435e+00, -6.9910e-01,  1.6830e-01,
        -9.7990e+00, -1.6850e+00,  4.2969e+00,  1.0471e+00, -3.7348e+00,
         1.4172e-03, -8.9905e+00,  1.9579e+00, -1.5481e+00, -7.7084e-01,
        -1.9058e+00,  1.5775e+00, -4.2176e+00, -4.4065e+00, -4.8470e+00,
         2.6566e+00, -6.9663e+00,  5.5089e+00, -3.0235e+00,  3.2943e+00,
        -8.1671e-01, -7.1863e-01,  6.5320e-01,  2.8318e+00,  2.1558e+00,
        -2.7233e+00, -1.3381e+00, -8.1541e+00, -5.0592e+00,  2.5273e+00,
        -7.7108e+00,  9.1387e+00, -5.1489e-01,  7.0222e+00,  1.3050e+00,
         5.3617e+00, -6.4973e+00, -3.7470e+00,  1.9152e+00, -6.4446e+00,
        -5.6579e+00, -4.1564e+00, -7.2266e+00, -1.7032e+00, -2.4363e+00,
        -7.6045e+00, -3.9795e+00,  4.5981e+00, -1.1

Función que convierte los datos textales en tensores w2v

In [195]:
def w2vify(batch):
    etiquetes = torch.LongTensor([comentario["coarse_label"] for comentario in batch])
    comentarios = torch.stack([to_w2v(tokenizer.tokenize(comentario["text"])) for comentario in batch])
    return etiquetes, comentarios

train_loader = DataLoader(ds_train, batch_size=16, collate_fn=w2vify)
test_loader = DataLoader(ds_test, batch_size=16, collate_fn=w2vify)

In [196]:
net = torch.nn.Sequential(
    torch.nn.Linear(300, 6),
    torch.nn.LogSoftmax(dim=1)
)

In [197]:
train_epoch(net, train_loader, epoch_size=5452)

800: acc=0.51
1600: acc=0.580625
2400: acc=0.6025
3200: acc=0.6221875
4000: acc=0.64075
4800: acc=0.6504166666666666


(0.05915870022791235, 0.6555392516507703)

Como podemos observar los resultados son ligeramente mejores con BoW que con Words2Vec